In [80]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import load_img

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.applications.xception import decode_predictions

from keras import layers


from tqdm.auto import tqdm 
import time

In [81]:
## Prep

In [82]:
input_size = 299

In [83]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    './clothing-dataset-small/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(input_size, input_size),
    shuffle=True
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomZoom(0.1),
])

def augment_and_preprocess(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 3068 files belonging to 10 classes.


In [84]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    './clothing-dataset-small/validation',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(input_size, input_size),
    shuffle=False
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomZoom(0.1),
])

def augment_and_preprocess(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

val_ds = val_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 341 files belonging to 10 classes.


In [85]:
## Build Model

In [86]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100, drop_rate=0.5):

    base_model = Xception(
        weights='imagenet', 
        include_top=False, 
        input_shape=(input_size, input_size, 3)
    )
    base_model.trainable = False
    
    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False) 
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(drop_rate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)
    model = keras.Model(inputs, outputs)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [8]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v5_{epoch:02d}_{val_accuracy:.3f}.keras', 
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [9]:
drop_rate = 0.2
learning_rate = 0.0005
size=50

model = make_model(
    input_size=299,
    learning_rate=learning_rate,
    size_inner=size,
    drop_rate=drop_rate
)
history = model.fit(
    train_ds, 
    epochs=5, 
    validation_data=val_ds,
    callbacks=[checkpoint]
)

Epoch 1/5
32/96 ━━━━━━━━━━━━━━━━━━━━ 1:11 1s/step - accuracy: 0.3313 - loss: 2.0009

KeyboardInterrupt: 

```
# Without Data Aug
Epoch 1/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 31s 314ms/step - accuracy: 0.5522 - loss: 1.3894 - val_accuracy: 0.7801 - val_loss: 0.6842
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 310ms/step - accuracy: 0.7555 - loss: 0.6901 - val_accuracy: 0.8123 - val_loss: 0.5758
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 311ms/step - accuracy: 0.8080 - loss: 0.5301 - val_accuracy: 0.8006 - val_loss: 0.5855
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 317ms/step - accuracy: 0.8429 - loss: 0.4260 - val_accuracy: 0.7977 - val_loss: 0.5529
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 31s 324ms/step - accuracy: 0.8910 - loss: 0.3264 - val_accuracy: 0.8211 - val_loss: 0.5373
Epoch 6/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 30s 316ms/step - accuracy: 0.9096 - loss: 0.2716 - val_accuracy: 0.8211 - val_loss: 0.5518
Epoch 7/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 32s 332ms/step - accuracy: 0.9366 - loss: 0.2115 - val_accuracy: 0.8006 - val_loss: 0.5800
```

### Using the Model

In [87]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    './clothing-dataset-small/test',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(input_size, input_size),
    shuffle=False
)

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomZoom(0.1),
])

def augment_and_preprocess(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label

test_ds = test_ds.map(augment_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

Found 372 files belonging to 10 classes.


In [88]:
model = keras.models.load_model('xception_v5_14_0.897.keras')

In [89]:
model.evaluate(test_ds)

12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8946 - loss: 0.3231


[0.2729645073413849, 0.9059139490127563]

In [90]:

path = 'clothing-dataset-small/test/longsleeve/0f4d9494-612a-48f8-be05-e47f4fc54d54.jpg'


In [91]:
img = load_img(path, target_size=(299, 299))


In [92]:
import numpy as np

x = np.array(img)
X = np.array([x])
X.shape

(1, 299, 299, 3)

In [93]:
X = preprocess_input(X)

In [94]:
pred = model.predict(X)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step


array([[ 1.3876281 , -4.026394  ,  4.76545   ,  2.6195517 , -2.0764413 ,
         1.6954165 , -5.9548235 , -1.2751677 , -0.16200481,  0.3598975 ]],
      dtype=float32)

In [95]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]


In [96]:
res = dict(zip(classes, pred[0]))
res

{'dress': 1.3876281,
 'hat': -4.026394,
 'longsleeve': 4.76545,
 'outwear': 2.6195517,
 'pants': -2.0764413,
 'shirt': 1.6954165,
 'shoes': -5.9548235,
 'shorts': -1.2751677,
 'skirt': -0.16200481,
 't-shirt': 0.3598975}

In [97]:
max(res.items(), key=lambda i: i[1])

('longsleeve', 4.76545)

## TF - Lite 

In [98]:
tf.__version__

'2.17.1'

In [99]:
model_path = 'xception_v5_14_0.897.keras'
model = keras.models.load_model(model_path)

In [100]:
#!wget http://bit.ly/mlbookcamp-pants -O dress.jpg

In [115]:
img = load_img('pants.jpg', target_size=(299, 299))
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)
X.shape

(1, 299, 299, 3)

In [116]:
preds = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


In [117]:
dict(zip(classes, preds[0]))

{'dress': -2.8748555,
 'hat': -2.6079617,
 'longsleeve': -2.2718732,
 'outwear': -1.0805153,
 'pants': 8.99109,
 'shirt': -0.52593803,
 'shoes': -4.0571136,
 'shorts': 4.15996,
 'skirt': -1.6107886,
 't-shirt': -2.6749864}

In [118]:
import tensorflow.lite as tflite

In [119]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('xception_v5_14_0.897.keras.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/9j/dq11054j6sg0m5lt67ct6fr40000gq/T/tmpu9uaefeh/assets


INFO:tensorflow:Assets written to: /var/folders/9j/dq11054j6sg0m5lt67ct6fr40000gq/T/tmpu9uaefeh/assets


Saved artifact at '/var/folders/9j/dq11054j6sg0m5lt67ct6fr40000gq/T/tmpu9uaefeh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_layer_9')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13544385296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544383376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544383568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544383760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544384336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544382032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544379728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544380688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544380304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544375888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13544381

W0000 00:00:1733736396.820627 1635341 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1733736396.820636 1635341 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-12-09 19:26:36.820752: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/9j/dq11054j6sg0m5lt67ct6fr40000gq/T/tmpu9uaefeh
2024-12-09 19:26:36.825167: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-09 19:26:36.825179: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/9j/dq11054j6sg0m5lt67ct6fr40000gq/T/tmpu9uaefeh
2024-12-09 19:26:36.870538: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-09 19:26:37.163160: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/9j/dq11054j6sg0m5lt67ct6fr40000gq/T/tmpu9uaefeh
2024-12-09 19:26:37.231908: I tensorflow/cc/saved_model/loader.cc:

In [120]:
interpreter = tflite.Interpreter(model_path='xception_v5_14_0.897.keras.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [121]:
preds

array([[-2.8748581, -2.6079607, -2.271874 , -1.0805144,  8.991091 ,
        -0.5259384, -4.0571146,  4.1599646, -1.6107898, -2.674987 ]],
      dtype=float32)

In [122]:
res = dict(zip(classes, preds[0]))
res

{'dress': -2.8748581,
 'hat': -2.6079607,
 'longsleeve': -2.271874,
 'outwear': -1.0805144,
 'pants': 8.991091,
 'shirt': -0.5259384,
 'shoes': -4.0571146,
 'shorts': 4.1599646,
 'skirt': -1.6107898,
 't-shirt': -2.674987}

In [123]:
max(res.items(), key=lambda x: x[1])

('pants', 8.991091)

In [4]:
!pip install keras-image-helper
!pip install tflite-runtime

ERROR: Could not find a version that satisfies the requirement tflite-runtime (from versions: none)
ERROR: No matching distribution found for tflite-runtime


In [125]:
# Simplifying.. 

In [126]:
import tensorflow.lite as tflite

from keras_image_helper import create_preprocessor
preprocessor = create_preprocessor('xception', target_size=(299, 299))

url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url) 

In [127]:
preds = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


In [128]:
res = dict(zip(classes, preds[0]))
res


{'dress': -2.8748555,
 'hat': -2.6079617,
 'longsleeve': -2.2718732,
 'outwear': -1.0805153,
 'pants': 8.99109,
 'shirt': -0.52593803,
 'shoes': -4.0571136,
 'shorts': 4.15996,
 'skirt': -1.6107886,
 't-shirt': -2.6749864}

In [129]:
max(res.items(), key=lambda x: x[1])

('pants', 8.99109)

In [12]:
# Remove dependency
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
ERROR: Could not find a version that satisfies the requirement tflite_runtime (from versions: none)
ERROR: No matching distribution found for tflite_runtime


In [130]:
# Use nbconvert and rename to lambda_function.py

### Test 

In [13]:
import lambda_function
url = 'http://bit.ly/mlbookcamp-pants'


In [14]:
lambda_function.predict(url)

{'dress': -2.8748581,
 'hat': -2.6079607,
 'longsleeve': -2.271874,
 'outwear': -1.0805144,
 'pants': 8.991091,
 'shirt': -0.5259384,
 'shoes': -4.0571146,
 'shorts': 4.1599646,
 'skirt': -1.6107898,
 't-shirt': -2.674987}

In [15]:
event = {'url': url}
lambda_function.lambda_handler(event)

{'dress': -2.8748581,
 'hat': -2.6079607,
 'longsleeve': -2.271874,
 'outwear': -1.0805144,
 'pants': 8.991091,
 'shirt': -0.5259384,
 'shoes': -4.0571146,
 'shorts': 4.1599646,
 'skirt': -1.6107898,
 't-shirt': -2.674987}

In [10]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

ModuleNotFoundError: No module named 'tflite_runtime'

In [16]:
!python test.py

{'dress': -2.874859094619751, 'hat': -2.607961416244507, 'longsleeve': -2.2718729972839355, 'outwear': -1.080513596534729, 'pants': 8.991092681884766, 'shirt': -0.5259398818016052, 'shoes': -4.0571136474609375, 'shorts': 4.159965991973877, 'skirt': -1.6107923984527588, 't-shirt': -2.674988269805908}
